In [23]:
from biom import load_table
import numpy as np
import pandas as pd
import seaborn as sns
from patsy import dmatrix
import os
%matplotlib inline

In [24]:
!ls ../data

File_1_TaoDing_microbiome.biom
File_2_TaoDing_Microbiome_taxonomy.tsv
File_3_FerretMicrobiomeMetadata.xlxs.xlsx
~$File_3_FerretMicrobiomeMetadata.xlxs.xlsx


In [25]:
fname = '../data/File_1_TaoDing_microbiome.biom'
table = load_table(fname)

In [26]:
table

7504 x 1096 <class 'biom.table.Table'> with 65829 nonzero entries (0% dense)

In [27]:
obs_filter = lambda v, i, m: (v>0).sum()>20
table.filter(obs_filter, axis='observation')

427 x 1096 <class 'biom.table.Table'> with 50968 nonzero entries (10% dense)

In [28]:
fname = 'File_3_FerretMicrobiomeMetadata.xlxs.xlsx'
fname = os.path.join('../data/', fname)
metadata = pd.read_excel(fname)

# Handle batch metadata

In [29]:
len(list(metadata.groupby('sampleID')))

811

In [30]:
id_counts = metadata.sampleID.value_counts()
batches = id_counts.loc[id_counts>1].index

In [31]:
batch_metadata = metadata.set_index('sampleID').loc[batches]

In [32]:
batch_metadata = batch_metadata.reset_index()

In [33]:
len(batch_metadata.sampleID.value_counts())

216

In [34]:
batch_design = dmatrix("sampleID + C(runID, Treatment('batch'))", 
                        batch_metadata, return_type='dataframe')

In [72]:
sample_ids = metadata.sampleID.unique()
id_lookup = pd.Series(np.arange(len(sample_ids)), index=sample_ids)
per_sample_ids = batch_metadata.sampleID.apply(lambda x: id_lookup[x]).values + 1

run_ids = batch_metadata.runID.unique()
id_lookup = pd.Series(np.arange(len(run_ids)), index=run_ids)
per_sample_batch_ids = batch_metadata.runID.apply(lambda x: id_lookup[x]).values + 1

In [63]:
# metadata.loc[metadata.Control, 'Days'] = 0
# TODO: may need to tinker with Days and Pre_Post
metadata['Days'] = metadata['Days'].astype(np.float64)
metadata['Pre_Post'] = metadata['Days'] > 0

In [64]:
main_design = dmatrix("Days + SampleType",metadata, return_type='dataframe')

# Subject specific intercepts

In [53]:
animal_ids = metadata.AnimalID.unique()

In [60]:
id_lookup = pd.Series(np.arange(len(animal_ids)), index=animal_ids)
per_sample_subject_ids = metadata.AnimalID.apply(lambda x: id_lookup[x]).values

# Actual modeling

We will adopt a Negative Binomial linear mixed effects models for this context.

Our model is given as follows

$\lambda_{ijkl} = x_i \cdot \beta_j + \gamma_k + \alpha_l$

$y_{ijkl} \sim NB(\lambda_{ijkl})$

Where x_i includes covariates `Days` and `SampleType`, $\gamma_k$ corresponds to the batch specific bias and $\alpha_l$ corresponds to the random intercept for each ferret.

See this document for more details on linear mixed models
http://jakewestfall.org/misc/SorensenEtAl.pdf

In [65]:
metadata

,sampleID,SeqID,runID,rundate,Experiment,Exp_short,SampleType,AnimalID,Days,Control,Pre_Post
0,14NW012029,14NW012029.20180509,20180509,20180509,CB1014,14,NW,2029,1.0,False,True
1,14NW012029,14NW012029.20180323,20180323,20180323,CB1014,14,NW,2029,1.0,False,True
2,14NW012030,14NW012030.20180323,20180323,20180323,CB1014,14,NW,2030,1.0,False,True
3,14NW012030,14NW012030.20180509,20180509,20180509,CB1014,14,NW,2030,1.0,False,True
4,14NW012031,14NW012031.20180509,20180509,20180509,CB1014,14,NW,2031,1.0,False,True
...,...,...,...,...,...,...,...,...,...,...,...
1054,7NW21852,7NW21852A.batch,batch,20180712,CB1007,7,NW,2185,2.0,False,True
1055,7NW21852,7NW21852B.batch,batch,20180712,CB1007,7,NW,2185,2.0,False,True
1056,7NW21862,7NW21862.batch,batch,20180712,CB1007,7,NW,2186,2.0,False,True
1057,8NW24243,8NW24243.batch,batch,20180712,CB1008,8,NW,2424,3.0,False,True


In [ ]:
per_sample_batch_ids